In [1]:
import geopandas as gpd
import momepy as mm
import libpysal
import pandas as pd
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import Polygon, MultiPolygon
from shapely import wkt
from shapely.ops import linemerge, unary_union, polygonize

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

In [10]:
%%time
tess = gpd.read_file('data/geometry_umm.gpkg', layer="encl_tess").dropna()
buildings = gpd.read_file('data/geometry_umm.gpkg', layer="buildings").dropna()
blocks = gpd.read_file('data/geometry_umm.gpkg', layer='blocks')

CPU times: total: 10.6 s
Wall time: 10.6 s


In [11]:
tess['tID'] = range(len(tess))

In [12]:
tess = tess.rename_geometry("tessellation").merge(buildings[["uID", "geometry","height"]].rename_geometry("buildings"), on="uID", how="left")

In [14]:
tess

,uID,eID,bID,nID,tID,tessellation,buildings,height
0,1814,0,0.0,28,0,"POLYGON ((413453.976 3390496.547, 413453.909 3...","POLYGON Z ((413488.104 3390520.045 6.000, 4134...",6.0
1,10316,0,0.0,28,1,"POLYGON ((413460.791 3390535.393, 413460.203 3...","POLYGON Z ((413506.478 3390528.655 21.000, 413...",21.0
2,10319,0,0.0,28,2,"POLYGON ((413479.709 3390559.201, 413479.606 3...","POLYGON Z ((413522.207 3390553.017 21.000, 413...",21.0
3,11556,0,0.0,2052,3,"POLYGON ((413481.440 3390644.908, 413481.406 3...","POLYGON Z ((413492.534 3390669.044 24.000, 413...",24.0
4,132,0,0.0,1701,4,"POLYGON ((413499.408 3390950.183, 413499.294 3...","POLYGON Z ((413509.930 3390954.680 3.000, 4135...",3.0
...,...,...,...,...,...,...,...,...
16432,8528,6642,3694.0,20161,16432,"POLYGON ((413233.422 3390976.899, 413245.608 3...","POLYGON Z ((413249.074 3390991.545 18.000, 413...",18.0
16433,11033,6649,3695.0,20295,16433,"POLYGON ((413431.265 3393271.437, 413430.047 3...","POLYGON Z ((413482.216 3393302.620 21.000, 413...",21.0
16434,12688,6650,2589.0,20268,16434,"POLYGON ((413455.376 3393493.685, 413458.542 3...","POLYGON Z ((413462.518 3393502.008 30.000, 413...",30.0
16435,11023,6651,3579.0,20270,16435,"POLYGON ((413447.358 3393259.226, 413471.664 3...","POLYGON Z ((413490.201 3393273.970 21.000, 413...",21.0


In [66]:
blg = tess.set_geometry('buildings').drop_duplicates('uID')
blg

,uID,eID,bID,nID,tID,tessellation,buildings,height,stcOri,stbOri,...,sicCAR,sicFAR,mtbAli,mtbNDi,mtcWNe,mdcAre,licGDe,ltcWRB,stcSAl,nodeID
0,1814,0,0.0,28,0,"POLYGON ((413453.976 3390496.547, 413453.909 3...","POLYGON Z ((413488.104 3390520.045 6.000, 4134...",6.0,1.929700,25.521722,...,0.027234,0.054468,3.533971,11.611677,0.005980,21206.548755,0.734619,0.000009,25.438274,45.0
1,10316,0,0.0,28,1,"POLYGON ((413460.791 3390535.393, 413460.203 3...","POLYGON Z ((413506.478 3390528.655 21.000, 413...",21.0,30.908144,27.538281,...,0.077110,0.539771,2.675848,15.126716,0.008798,45588.458569,0.963148,0.000033,3.540170,45.0
2,10319,0,0.0,28,2,"POLYGON ((413479.709 3390559.201, 413479.606 3...","POLYGON Z ((413522.207 3390553.017 21.000, 413...",21.0,30.941455,25.659711,...,0.053695,0.375866,3.430163,36.029675,0.008261,64455.316359,0.934488,0.000035,3.573482,45.0
3,11556,0,0.0,2052,3,"POLYGON ((413481.440 3390644.908, 413481.406 3...","POLYGON Z ((413492.534 3390669.044 24.000, 413...",24.0,14.750400,24.572171,...,0.026692,0.213532,6.307824,127.264222,0.008990,61460.458254,0.837727,0.000051,13.983532,2395.0
4,132,0,0.0,1701,4,"POLYGON ((413499.408 3390950.183, 413499.294 3...","POLYGON Z ((413509.930 3390954.680 3.000, 4135...",3.0,42.437219,31.073709,...,0.010260,0.010260,1.581710,15.212978,0.008328,24294.437426,0.554848,0.000042,29.299289,2060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16422,2478,6620,3687.0,19638,16422,"POLYGON ((412419.683 3392298.746, 412435.602 3...","POLYGON Z ((412443.326 3392307.147 6.000, 4124...",6.0,32.228712,30.233279,...,0.192262,0.384524,1.874290,12.414926,0.019168,4898.057723,3.330393,0.000242,9.866882,14364.0
16424,10761,6622,3688.0,19657,16424,"POLYGON ((412451.443 3392186.546, 412467.367 3...","POLYGON Z ((412475.456 3392216.010 21.000, 412...",21.0,31.677262,33.474946,...,0.350861,2.456024,2.590536,18.493013,0.020406,11731.748953,3.225671,0.000242,0.000053,14369.0
16431,3074,6639,3693.0,20069,16431,"POLYGON ((413164.243 3393851.806, 413132.543 3...","POLYGON Z ((413151.979 3393859.823 6.000, 4131...",6.0,20.031683,23.373054,...,0.103197,0.206394,2.746993,17.064368,0.018940,2075.054855,2.514806,0.000184,10.117025,13304.0
16432,8528,6642,3694.0,20161,16432,"POLYGON ((413233.422 3390976.899, 413245.608 3...","POLYGON Z ((413249.074 3390991.545 18.000, 413...",18.0,25.345461,28.469113,...,0.298462,1.790775,0.809373,13.357651,0.030135,5863.174739,2.384406,0.000312,25.292453,3449.0


In [17]:
# blg['height'] = blg.height.fillna(3)

In [15]:
# blg.loc[blg["height"] < 3, "height"] = 3 #substitute everything below 3m with 3m of height

In [17]:
tess[['uID','eID','bID','nID','tID','tessellation']].to_file('data/geometry_umm.gpkg', layer="encl_tess", driver="GPKG")

## measure characters

In [17]:
blg['sdbAre'] = mm.Area(blg).series
blg['sdbVol'] = mm.Volume(blg, 'height', 'sdbAre').series #only if you have building height
blg['sdbPer'] = mm.Perimeter(blg).series
blg['sdbCoA'] = mm.CourtyardArea(blg, 'sdbAre').series
blg['ssbFoF'] = mm.FormFactor(blg, 'sdbVol', heights='height').series #only if you have building height
blg['ssbVFR'] = mm.VolumeFacadeRatio(blg, 'height', 'sdbVol', 'sdbPer').series #only if you have building height
blg['ssbCCo'] = mm.CircularCompactness(blg, 'sdbAre').series
blg['ssbCor'] = mm.Corners(blg).series
blg['ssbSqu'] = mm.Squareness(blg).series
blg['ssbERI'] = mm.EquivalentRectangularIndex(blg, 'sdbAre', 'sdbPer').series
blg['ssbElo'] = mm.Elongation(blg).series

  0%|          | 0/13981 [00:00<?, ?it/s]

  0%|          | 0/13981 [00:00<?, ?it/s]

In [18]:
cencon = mm.CentroidCorners(blg)
blg['ssbCCM'] = cencon.mean
blg['ssbCCD'] = cencon.std

  0%|          | 0/13981 [00:00<?, ?it/s]

In [19]:
blg['stbOri'] = mm.Orientation(blg).series

  0%|          | 0/13981 [00:00<?, ?it/s]

In [20]:
tess['stcOri'] = mm.Orientation(tess).series

  0%|          | 0/16437 [00:00<?, ?it/s]

In [21]:
tess = tess.merge(blg[['tID', 'stbOri']], on='tID', how='left')

In [22]:
tess['stbCeA'] = (tess['stbOri'] - tess['stcOri']).abs()

In [23]:
tess['sdcLAL'] = mm.LongestAxisLength(tess).series
tess['sdcAre'] = mm.Area(tess).series
tess['sscCCo'] = mm.CircularCompactness(tess, 'sdcAre').series
tess['sscERI'] = mm.EquivalentRectangularIndex(tess, 'sdcAre').series
tess['sicCAR'] = mm.AreaRatio(tess.set_geometry('tessellation'), tess.set_geometry('buildings'), 'sdcAre', tess.buildings.area, 'uID').series

In [24]:
tess['sicFAR'] = mm.AreaRatio(tess.set_geometry('tessellation'), tess.set_geometry('buildings'), 'sdcAre', (tess.buildings.area*tess['height'])/3, 'tID').series 

In [25]:
# tess.sicCAR[tess.sicCAR > 1] = 1

In [26]:
blg["mtbSWR"] = mm.SharedWallsRatio(blg).series

In [27]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tess, ids="tID", geom_col = 'tessellation')

In [28]:
tess["mtbAli"] = mm.Alignment(tess.set_geometry("buildings"), queen_1, "tID", "stbOri").series

  0%|          | 0/16437 [00:00<?, ?it/s]

In [29]:
tess["mtbNDi"] = mm.NeighborDistance(tess.set_geometry("buildings"), queen_1, "tID").series

  0%|          | 0/16437 [00:00<?, ?it/s]

In [30]:
tess["mtcWNe"] = mm.Neighbors(tess, queen_1, "tID", weighted=True).series
tess["mdcAre"] = mm.CoveredArea(tess, queen_1, "tID").series

  0%|          | 0/16437 [00:00<?, ?it/s]

  0%|          | 0/16437 [00:00<?, ?it/s]

In [31]:
blg_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blg)

In [32]:
blg["libNCo"] = mm.Courtyards(blg, blg_q1).series
blg["ldbPWL"] = mm.PerimeterWall(blg, blg_q1).series
 
blocks["ldkAre"] = mm.Area(blocks).series
blocks["ldkPer"] = mm.Perimeter(blocks).series
blocks["lskCCo"] = mm.CircularCompactness(blocks, "ldkAre").series
blocks["lskERI"] = mm.EquivalentRectangularIndex(blocks, "ldkAre", "ldkPer").series
blocks["lskCWA"] = mm.CompactnessWeightedAxis(blocks, "ldkAre", "ldkPer").series
blocks["ltkOri"] = mm.Orientation(blocks).series
 
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blocks, ids="bID")
 
blocks["ltkWNB"] = mm.Neighbors(blocks, blo_q1, "bID", weighted=True).series
blocks["likWBB"] = mm.Count(blocks, blg, "bID", "bID", weighted=True).series

  0%|          | 0/13981 [00:00<?, ?it/s]

  0%|          | 0/13981 [00:00<?, ?it/s]

  0%|          | 0/13981 [00:00<?, ?it/s]

  0%|          | 0/3696 [00:00<?, ?it/s]

  0%|          | 0/3696 [00:00<?, ?it/s]

In [33]:
#Save data to parquets

In [34]:
tess.drop(columns=['tessellation', 'buildings']).to_parquet('data/tess_data.parquet')
blg.drop(columns=['tessellation', 'buildings']).to_parquet('data/blg_data.parquet')
blocks.drop(columns='geometry').to_parquet('data/blocks_data.parquet')

In [35]:
queen3 = mm.sw_high(k=3, weights=queen_1)
queen1 = queen_1

In [36]:
blg_queen = blg_q1

In [37]:
blg['ltbIBD'] = mm.MeanInterbuildingDistance(tess.set_geometry("buildings"), queen1, 'tID').series

Computing mean interbuilding distances...


  0%|          | 0/16437 [00:00<?, ?it/s]

In [38]:
blg['ltcBuA'] = mm.BuildingAdjacency(tess.set_geometry("buildings"), queen3, 'tID').series

Calculating spatial weights...
Spatial weights ready...


Calculating adjacency:   0%|          | 0/16437 [00:00<?, ?it/s]

In [39]:
tess['licGDe'] = mm.Density(tess.set_geometry('tessellation'), (tess.buildings.area*tess['height'])/3, queen3, 'tID', 'sdcAre').series

  0%|          | 0/16437 [00:00<?, ?it/s]

In [40]:
tess['ltcWRB'] = mm.BlocksCount(tess.set_geometry("tessellation"), 'bID', queen3, 'tID').series 

  0%|          | 0/16437 [00:00<?, ?it/s]

In [41]:
#Save data to parquets and spatial weights matrices to gal files

In [42]:
tess.drop(columns=['tessellation', 'buildings']).to_parquet('data/tess_data.parquet')
blg.drop(columns=['tessellation', 'buildings']).to_parquet('data/blg_data.parquet')

In [43]:
fo = libpysal.io.open('data/queen1.gal', 'w')
fo.write(queen1)
fo.close()
 
fo = libpysal.io.open('data/queen3.gal', 'w')
fo.write(queen3)
fo.close()
 
fo = libpysal.io.open('data/blg_queen.gal', 'w')
fo.write(blg_queen)
fo.close()

In [44]:
streets = gpd.read_file('data/geometry_umm.gpkg', layer="streets")

In [45]:
streets["sdsLen"] = mm.Perimeter(streets).series
tess["stcSAl"] = mm.StreetAlignment(tess, streets, "stcOri", "nID").series
blg["stbSAl"] = mm.StreetAlignment(blg, streets, "stbOri", "nID").series

profile = mm.StreetProfile(streets, tess.set_geometry("buildings"), heights=tess.height) #only if you have building height
streets["sdsSPW"] = profile.w #only if you have building height
streets["sdsSPH"] = profile.h #only if you have building height
streets["sdsSPR"] = profile.p #only if you have building height
streets["sdsSPO"] = profile.o #only if you have building height
streets["sdsSWD"] = profile.wd #only if you have building height
streets["sdsSHD"] = profile.hd #only if you have building height
 
streets["sssLin"] = mm.Linearity(streets).series
streets["sdsAre"] = mm.Reached(streets, tess, "nID", "nID", mode="sum", values="sdcAre").series
streets["sisBpM"] = mm.Count(streets, blg, "nID", "nID", weighted=True).series

  0%|          | 0/20324 [00:00<?, ?it/s]

In [46]:
tess.drop(columns=['tessellation', 'buildings']).to_parquet('data/tess_data.parquet')
blg.drop(columns=['tessellation', 'buildings']).to_parquet('data/blg_data.parquet')
streets.drop(columns='geometry').to_parquet('data/streets_data.parquet')

In [47]:
str_q1 = libpysal.weights.contiguity.Queen.from_dataframe(streets)
 
streets["misRea"] = mm.Reached(streets, tess, "nID", "nID", spatial_weights=str_q1, mode="count").series
streets["mdsAre"] = mm.Reached(streets, tess, "nID", "nID", spatial_weights=str_q1, mode="sum").series

  0%|          | 0/20324 [00:00<?, ?it/s]

  0%|          | 0/20324 [00:00<?, ?it/s]

In [48]:
graph = mm.gdf_to_nx(streets)
 
print("node degree")
graph = mm.node_degree(graph)
 
print("subgraph")
graph = mm.subgraph(
    graph,
    radius=5,
    meshedness=True,
    cds_length=False,
    mode="sum",
    degree="degree",
    length="mm_len",
    mean_node_degree=False,
    proportion={0: True, 3: True, 4: True},
    cyclomatic=False,
    edge_node_ratio=False,
    gamma=False,
    local_closeness=True,
    closeness_weight="mm_len",
)
print("cds length")
graph = mm.cds_length(graph, radius=3, name="ldsCDL")
 
print("clustering")
graph = mm.clustering(graph, name="xcnSCl")
 
print("mean_node_dist")
graph = mm.mean_node_dist(graph, name="mtdMDi")
 
nodes, edges, sw = mm.nx_to_gdf(graph, spatial_weights=True)

node degree
subgraph


  0%|          | 0/14489 [00:00<?, ?it/s]

cds length


  0%|          | 0/14489 [00:00<?, ?it/s]

clustering
mean_node_dist


  0%|          | 0/14489 [00:00<?, ?it/s]

In [49]:
print("saving")
nodes.to_file('data/geometry_umm.gpkg', layer="nodes", driver="GPKG")
edges.to_file('data/geometry_umm.gpkg', layer="edges", driver="GPKG")

saving


In [50]:
fo = libpysal.io.open('data/UAPnodes.gal', "w")
fo.write(sw)
fo.close()
 
edges_w3 = mm.sw_high(k=3, gdf=edges)
edges["ldsMSL"] = mm.SegmentsLength(edges, spatial_weights=edges_w3, mean=True).series
 
edges["ldsRea"] = mm.Reached(edges, tess, "nID", "nID", spatial_weights=edges_w3).series
edges["ldsRea"] = mm.Reached(edges, tess, "nID", "nID", spatial_weights=edges_w3, mode="sum", values="sdcAre").series
 
nodes_w5 = mm.sw_high(k=5, weights=sw)
nodes["lddNDe"] = mm.NodeDensity(nodes, edges, nodes_w5).series
nodes["linWID"] = mm.NodeDensity(nodes, edges, nodes_w5, weighted=True, node_degree="degree").series

  0%|          | 0/20324 [00:00<?, ?it/s]

  0%|          | 0/20324 [00:00<?, ?it/s]

  0%|          | 0/20324 [00:00<?, ?it/s]

  0%|          | 0/14489 [00:00<?, ?it/s]

  0%|          | 0/14489 [00:00<?, ?it/s]

In [51]:
edges['nID'] = edges['nID'].astype(float)
blg['nID'] = blg['nID'].astype(float)

In [52]:
blg["nodeID"] = mm.get_node_id(blg, nodes, edges, "nodeID", "nID")

  0%|          | 0/13981 [00:00<?, ?it/s]

In [53]:
tess = tess.merge(blg[["tID", "nodeID"]], on="tID", how="left")
 
nodes_w3 = mm.sw_high(k=3, weights=sw)
 
nodes["lddRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", nodes_w3).series
nodes["lddARe"] = mm.Reached(nodes, tess, "nodeID", "nodeID", nodes_w3, mode="sum", values="sdcAre").series
 
nodes["sddAre"] = mm.Reached(nodes, tess, "nodeID", "nodeID", mode="sum", values="sdcAre").series
nodes["midRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", spatial_weights=sw).series
nodes["midAre"] = mm.Reached(nodes, tess, "nodeID", "nodeID", spatial_weights=sw, mode="sum", values="sdcAre").series
 
nodes.rename(
    columns={
        "degree": "mtdDeg",
        "meshedness": "lcdMes",
        "local_closeness": "lcnClo",
        "proportion_3": "linP3W",
        "proportion_4": "linP4W",
        "proportion_0": "linPDE",
    }, inplace=True
)

  0%|          | 0/14489 [00:00<?, ?it/s]

  0%|          | 0/14489 [00:00<?, ?it/s]

  0%|          | 0/14489 [00:00<?, ?it/s]

  0%|          | 0/14489 [00:00<?, ?it/s]

  0%|          | 0/14489 [00:00<?, ?it/s]

In [54]:
tess.drop(columns=['tessellation', 'buildings']).to_parquet('data/tess_data.parquet')
blg.drop(columns=['tessellation', 'buildings']).to_parquet('data/blg_data.parquet')
nodes.drop(columns='geometry').to_parquet('data/nodes_data.parquet')
edges.drop(columns='geometry').to_parquet('data/edges_data.parquet')

In [55]:
merged = tess.merge(blg.drop(columns=['uID','eID','bID','nID','tessellation','buildings','height','stbOri','nodeID']), on='tID', how='left')

In [56]:
merged['nID'] = merged['nID'].astype(float)
merged['eID'] = merged['eID'].astype(float)

In [57]:
merged = merged.merge(blocks.drop(columns='geometry'), on='bID', how='left')
merged = merged.merge(edges.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')

In [58]:
merged.rename(columns={'height': 'sdbHei'}, inplace=True)

In [59]:
merged.columns

Index(['uID', 'eID', 'bID', 'nID', 'tID', 'tessellation', 'buildings',
       'sdbHei', 'stcOri', 'stbOri', 'stbCeA', 'sdcLAL', 'sdcAre', 'sscCCo',
       'sscERI', 'sicCAR', 'sicFAR', 'mtbAli', 'mtbNDi', 'mtcWNe', 'mdcAre',
       'licGDe', 'ltcWRB', 'stcSAl', 'nodeID', 'sdbAre', 'sdbVol', 'sdbPer',
       'sdbCoA', 'ssbFoF', 'ssbVFR', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbERI',
       'ssbElo', 'ssbCCM', 'ssbCCD', 'mtbSWR', 'libNCo', 'ldbPWL', 'ltbIBD',
       'ltcBuA', 'stbSAl', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA',
       'ltkOri', 'ltkWNB', 'likWBB', 'sdsLen', 'sdsSPW', 'sdsSPH', 'sdsSPR',
       'sdsSPO', 'sdsSWD', 'sdsSHD', 'sssLin', 'sdsAre', 'sisBpM', 'misRea',
       'mdsAre', 'mm_len', 'cdsbool', 'node_start', 'node_end', 'ldsMSL',
       'ldsRea', 'mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'ldsCDL', 'xcnSCl', 'mtdMDi', 'lddNDe', 'linWID', 'lddRea', 'lddARe',
       'sddAre', 'midRea', 'midAre'],
      dtype='object')

In [60]:
#Clean columns to keep only data
primary = merged.drop(columns=['eID', 'bID', 'nID', 'mm_len', 'cdsbool', 'nodeID', 'node_start', 'node_end'])

In [61]:
primary.columns.to_list()

['uID',
 'tID',
 'tessellation',
 'buildings',
 'sdbHei',
 'stcOri',
 'stbOri',
 'stbCeA',
 'sdcLAL',
 'sdcAre',
 'sscCCo',
 'sscERI',
 'sicCAR',
 'sicFAR',
 'mtbAli',
 'mtbNDi',
 'mtcWNe',
 'mdcAre',
 'licGDe',
 'ltcWRB',
 'stcSAl',
 'sdbAre',
 'sdbVol',
 'sdbPer',
 'sdbCoA',
 'ssbFoF',
 'ssbVFR',
 'ssbCCo',
 'ssbCor',
 'ssbSqu',
 'ssbERI',
 'ssbElo',
 'ssbCCM',
 'ssbCCD',
 'mtbSWR',
 'libNCo',
 'ldbPWL',
 'ltbIBD',
 'ltcBuA',
 'stbSAl',
 'ldkAre',
 'ldkPer',
 'lskCCo',
 'lskERI',
 'lskCWA',
 'ltkOri',
 'ltkWNB',
 'likWBB',
 'sdsLen',
 'sdsSPW',
 'sdsSPH',
 'sdsSPR',
 'sdsSPO',
 'sdsSWD',
 'sdsSHD',
 'sssLin',
 'sdsAre',
 'sisBpM',
 'misRea',
 'mdsAre',
 'ldsMSL',
 'ldsRea',
 'mtdDeg',
 'lcdMes',
 'linP3W',
 'linP4W',
 'linPDE',
 'lcnClo',
 'ldsCDL',
 'xcnSCl',
 'mtdMDi',
 'lddNDe',
 'linWID',
 'lddRea',
 'lddARe',
 'sddAre',
 'midRea',
 'midAre']

In [62]:
primary.to_parquet('data/primary.pq')